# Preparing Dataset

In [2]:
import os
import json
import librosa 
import math
import numpy as np
from sklearn import model_selection
import keras
from keras.models import Sequential
from keras.layers import Dense

In [8]:
data_path="genres"
json_path="json_data.json"

In [5]:
SR=22050
dur=30
total_samples=SR*dur

In [6]:
def reverse_str(s):
    ans=""
    for i in range(len(s)):
        ans=s[i]+ans
    return ans

In [7]:
def prepare_data_mfcc(data_path,json_path,n_mfcc=13,n_fft=2048,hop_length=512,num_segments=10):
    
    data={
        "mapping":[],  # Mapping of genre with the index
        "mfcc":[],
        "labels":[]
    }
    
    for count,(dirpath,dirnames,filenames) in enumerate(os.walk(data_path)):
        if dirpath==data_path:
            continue
        else:
            
            # Extracting the genre
            
            gen=""
            dirgen=reverse_str(dirpath)
            for i in range(len(dirgen)):
                if(dirgen[i]=="\\"):
                   break
                else:
                   gen+=dirgen[i]
            print(reverse_str(gen))
            data["mapping"].append(reverse_str(gen))
            
            # Now for each genre exploring all of the files
    
            for file in filenames:
            
                # loading audio file
                
                file_path=os.path.join(dirpath,file)
                signal,sr=librosa.load(file_path,SR)
                
                # Chopping off into segments
                
                num_samples_per_segment=total_samples//num_segments
                num_mfcc_vec_seg=math.ceil(num_samples_per_segment/hop_length)
                
                for segment in range(num_segments):
                    
                    start_seg=num_samples_per_segment*segment
                    end_seg=start_seg+num_samples_per_segment
                    mfcc_seg=librosa.feature.mfcc(signal[start_seg:end_seg],sr=SR,n_fft=n_fft,n_mfcc=n_mfcc,hop_length=hop_length)
                    mfcc_seg=mfcc_seg.T 
                    
                    # Stroring the mfcc if it has the expected length
                    
                    if len(mfcc_seg)==num_mfcc_vec_seg:
                        data["mfcc"].append(mfcc_seg.tolist())
                        data["labels"].append(count-1) 
    
    with open(json_path,"w") as fp:
        json.dump(data,fp,indent=4)    

In [9]:
prepare_data_mfcc(data_path,json_path)

blues
classical
country
disco
hiphop
jazz
metal
pop
reggae
rock


In [10]:
json_file=open(json_path)
data=json.load(json_file)

In [11]:
# Confirming the shapes of data

print(len(data["mapping"]))
print(len(data["mfcc"]))
print(len(data["labels"])) 
print(len(data["mfcc"][0]))

10
9996
9996
130


In [12]:
# Train_Test Split

X_Train,X_Test,Y_Train,Y_Test=model_selection.train_test_split(data["mfcc"],data["labels"],test_size=0.2)
print("Size of Train_Data=",len(X_Train))
print("Size of Test_Data=",len(X_Test))

Size of Train_Data= 7996
Size of Test_Data= 2000


In [13]:
# Since CNN expects a 3rd dimension i.e the depth

np_arr_train=np.array(X_Train)
print(np_arr_train.shape)
np_arr_train=np_arr_train.reshape(7996,130,13,1)

np_arr_test=np.array(X_Test)
print(np_arr_test.shape)

np_arr_test=np_arr_test.reshape(2000,130,13,1)
print(np_arr_test.shape)

X_Train=np_arr_train
X_Test=np_arr_test

(7996, 130, 13)
(2000, 130, 13)
(2000, 130, 13, 1)


# Building the Architecture of the model

In [14]:
def build_architecture(data_shape):
    model=keras.Sequential()
    print(data_shape)
    # 1st Convolutional Layer
    
    model.add(keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=data_shape,data_format="channels_last"))
    model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same"))
    
    #2nd Convolutional Layer
    
    model.add(keras.layers.Conv2D(32,(3,3),activation="relu",data_format="channels_last"))
    model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same"))
    
    # Flattening the data
    
    model.add(keras.layers.Flatten())
    
    # Dense Layer
    
    model.add(keras.layers.Dense(32,activation="relu"))
    
    # Dropout
    
    model.add(keras.layers.Dropout(0.3))
    
    # Output Layer
    
    model.add(keras.layers.Dense(10,activation="softmax"))
    
    return model

In [15]:
model=build_architecture((130,13,1))

(130, 13, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



# Compiling the Model

In [16]:

optimizer=keras.optimizers.Adam(learning_rate=0.0001)

# I have used sparse categorical cross entropy because we don't have to provide labels as one hot encoding in this

model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [17]:
# Creating Validation Data from Train Data

X_Train,X_Valid,Y_Train,Y_Valid=model_selection.train_test_split(X_Train,Y_Train,test_size=0.2)

In [18]:
#  Training the CNN

model.fit(X_Train,Y_Train,validation_data=(X_Valid,Y_Valid),batch_size=64,epochs=100)

# Concentrate on validation_accuracy


Train on 6396 samples, validate on 1600 samples
Epoch 1/100
6396/6396 [==============================] - 8s 1ms/step - loss: 3.8498 - accuracy: 0.1417 - val_loss: 2.1443 - val_accuracy: 0.2131
Epoch 2/100
6396/6396 [==============================] - 8s 1ms/step - loss: 2.1261 - accuracy: 0.2104 - val_loss: 2.0208 - val_accuracy: 0.2550
Epoch 3/100
6396/6396 [==============================] - 8s 1ms/step - loss: 2.0551 - accuracy: 0.2319 - val_loss: 1.9779 - val_accuracy: 0.2556
Epoch 4/100
6396/6396 [==============================] - 8s 1ms/step - loss: 2.0121 - accuracy: 0.2461 - val_loss: 1.9237 - val_accuracy: 0.3081
Epoch 5/100
6396/6396 [==============================] - 7s 1ms/step - loss: 1.9853 - accuracy: 0.2609 - val_loss: 1.9155 - val_accuracy: 0.2969
Epoch 6/100
6396/6396 [==============================] - 8s 1ms/step - loss: 1.9421 - accuracy: 0.2703 - val_loss: 1.8615 - val_accuracy: 0.3338
Epoch 7/100
6396/6396 [==============================] - 8s 1ms/step - loss: 1.90

6396/6396 [==============================] - 7s 1ms/step - loss: 1.2016 - accuracy: 0.5377 - val_loss: 1.2520 - val_accuracy: 0.5437
Epoch 56/100
6396/6396 [==============================] - 7s 1ms/step - loss: 1.1758 - accuracy: 0.5521 - val_loss: 1.2607 - val_accuracy: 0.5469
Epoch 57/100
6396/6396 [==============================] - 7s 1ms/step - loss: 1.1837 - accuracy: 0.5442 - val_loss: 1.2599 - val_accuracy: 0.5450
Epoch 58/100
6396/6396 [==============================] - 7s 1ms/step - loss: 1.1568 - accuracy: 0.5530 - val_loss: 1.2732 - val_accuracy: 0.5469
Epoch 59/100
6396/6396 [==============================] - 8s 1ms/step - loss: 1.1497 - accuracy: 0.5624 - val_loss: 1.2712 - val_accuracy: 0.5419
Epoch 60/100
6396/6396 [==============================] - 8s 1ms/step - loss: 1.1365 - accuracy: 0.5624 - val_loss: 1.2786 - val_accuracy: 0.5337
Epoch 61/100
6396/6396 [==============================] - 7s 1ms/step - loss: 1.1469 - accuracy: 0.5527 - val_loss: 1.2536 - val_accuracy

# Predicting on Test Data

In [19]:
Y_Pred_onehot=model.predict(X_Test)

# Y_Pred single query is a 10 sized array so we need to take the maximum one

Y_Pred=[]

for i in range(len(Y_Pred_onehot)):
    predicted_index=0
    maxpred=0
    for j in range(10):
        if(Y_Pred_onehot[i][j]>maxpred):
            maxpred=Y_Pred_onehot[i][j]
            predicted_index=j
    Y_Pred.append(predicted_index)
    
Y_Pred

[5,
 2,
 4,
 8,
 5,
 5,
 3,
 8,
 4,
 1,
 3,
 2,
 8,
 9,
 9,
 0,
 5,
 6,
 4,
 3,
 9,
 2,
 4,
 5,
 5,
 0,
 7,
 6,
 0,
 5,
 0,
 9,
 4,
 1,
 6,
 4,
 9,
 0,
 3,
 2,
 1,
 2,
 0,
 1,
 8,
 5,
 2,
 6,
 8,
 8,
 9,
 1,
 9,
 4,
 8,
 0,
 3,
 2,
 4,
 5,
 3,
 6,
 7,
 1,
 2,
 0,
 6,
 0,
 1,
 9,
 6,
 6,
 2,
 9,
 0,
 7,
 3,
 5,
 1,
 2,
 3,
 7,
 4,
 0,
 3,
 8,
 4,
 7,
 8,
 0,
 7,
 6,
 4,
 4,
 9,
 2,
 3,
 2,
 0,
 2,
 4,
 3,
 8,
 8,
 7,
 0,
 3,
 4,
 4,
 8,
 3,
 5,
 2,
 4,
 6,
 0,
 2,
 8,
 0,
 9,
 0,
 3,
 8,
 0,
 4,
 7,
 6,
 6,
 8,
 0,
 5,
 0,
 8,
 0,
 7,
 4,
 5,
 0,
 5,
 4,
 2,
 3,
 9,
 3,
 5,
 6,
 2,
 4,
 3,
 4,
 4,
 1,
 2,
 1,
 0,
 8,
 8,
 0,
 3,
 2,
 1,
 0,
 7,
 9,
 5,
 0,
 9,
 2,
 0,
 9,
 8,
 1,
 7,
 3,
 7,
 7,
 6,
 1,
 2,
 7,
 9,
 2,
 0,
 1,
 1,
 9,
 1,
 6,
 5,
 2,
 9,
 3,
 6,
 5,
 6,
 9,
 1,
 1,
 3,
 5,
 7,
 7,
 6,
 4,
 0,
 1,
 4,
 7,
 2,
 4,
 3,
 4,
 3,
 7,
 7,
 3,
 1,
 1,
 5,
 4,
 2,
 4,
 9,
 5,
 8,
 8,
 3,
 7,
 6,
 4,
 1,
 8,
 7,
 4,
 0,
 7,
 1,
 2,
 8,
 7,
 7,
 1,
 0,
 4,
 6,
 0,
 4,
 2,
 7,
 9,


In [29]:
print("The actual genre is",data["mapping"][Y_Test[200]])
print("The predicted genre is",data["mapping"][Y_Pred[200]])

The actual genre is pop
The predicted genre is pop


In [31]:
test_error,test_accuracy=model.evaluate(X_Test,Y_Test)
print("The error and accuracy of the model on test set is",test_error,"and",test_accuracy)

2000/2000 [==============================] - 1s 270us/step
The error and accuracy of the model on test set is 1.2916027841567994 and 0.5839999914169312
